In [ ]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize 
import nltk
from collections import defaultdict
import pandas as pd
import operator
import matplotlib.pyplot as plt 
import nltk
import csv
from nltk.corpus import stopwords
import glob, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
%matplotlib inline

data = pd.read_csv('/home/oem/jin/python/after_prepro10yr.csv', encoding="CP949")


print(len(data.iloc[:,0]))



X = data.iloc[:,1].values
y = data.iloc[:,2].values
print(X)
print(y)
#y = data[2]


x1 = []

for sre in X:
    text = sre.rstrip()
    
    x1.append(text)
    

#print(len(x1))

from keras.preprocessing.text import Tokenizer


tokenizer = Tokenizer(num_words=8000)
# 그에 맞게 단어 인덱스를 구축
tokenizer.fit_on_texts(x1)

# 문자열을 인덱스의 리스트로 변환
sequences = tokenizer.texts_to_sequences(x1)
index = tokenizer.word_index
index_word = tokenizer.index_word
print(index["미세먼지"])
print(index_word[1])



encoded = tokenizer.texts_to_sequences(X)
dataset = []
labels = y
num_labels = len(labels)
for i,sentence in enumerate(encoded):
    dataset.append((sentence,y[i]))
#print(dataset)
from collections import Counter
counter_list = []
for label in labels:
    label_dic = list(filter(lambda x: x[1]==label, dataset))
    label_dic = [e[0] for e in label_dic]
    label_dic = [e for element in label_dic for e in element]
    label_dic = Counter(label_dic)
    counter_list.append(label_dic)



vocab = 8000

word_freq_dic = {}   ###### 문서별 단어 빈도수 계산 하기 
for word_idx in range(1,vocab):
    word_freq = []
    #print(word_idx)
    for i in range(len(labels)):
        word_freq.append((counter_list[i][word_idx]))
    word_freq_dic[word_idx] = word_freq
        


igm_dict = {}
wg_dict = {}
alpha = 7
for word_idx in range(1,vocab):
    sorted_list = sorted(word_freq_dic[word_idx],reverse=True)
    denom = 0
    for i in range(num_labels):
        denom += sorted_list[i]*(i+1)
        
        
    igm_dict[word_idx] = sorted_list[0]/denom
    wg_dict[word_idx] = 1+(alpha*igm_dict[word_idx])
    


idx_to_count = [e[0] for e in dataset]
idx_to_count = Counter([e for element in idx_to_count for e in element])
tf_term = {}
for word_idx in range(1,vocab):
    tf_term[word_idx] =  idx_to_count[word_idx] / vocab
    
tf_igm = {}
for word_idx in range(1,vocab):
    tf_igm[word_idx] = tf_term[word_idx]*wg_dict[word_idx]
    

    
lst = np.zeros((44491, 8000))


to = 0
for line in encoded:
    for x in line:
        
        result = tf_igm[x]
        
        lst[to][x] = result

    to += 1
    

decoding = pad_sequences(lst, padding='post', dtype='float32')

    
    

X_train, X_test, y_train, y_test = train_test_split(decoding,y, test_size=0.25, shuffle=True, random_state=25, stratify=y)



In [ ]:
##### deault ML capplication #######


svc = SVC()
LogReg = LogisticRegression()
nb = MultinomialNB()
rf = RandomForestClassifier()
xgb = XGBClassifier()
model = [svc, LogReg, nb, rf, xgb]


for m in model:
    m.fit(X_train, y_train)
    

for m in model:
    pred = m.predict(X_test)
    print('Acc for model', accuracy_score(y_test,pred))
    print('Confusion Matrix for model', confusion_matrix(y_test, pred))
    print('F1-score for model', classification_report(y_test, pred))
    
##### end of the defulat ML application ########

In [ ]:
##### gridsearchCV for SVM classifer 


svm_tf_igm = open('tf_igm_svm.txt', 'w')

parameters = [{"kernel":["rbf"], "gamma":[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05,0.1, 1, 5, 10, 20, 50, 100, 200, 500, 1000], "C":[0.001, 0.005, 0.01, 0.1, 1, 5, 10, 20, 30, 40, 50, 100, 200, 500, 700, 1000], "degree":[1, 2, 3, 4, 5]}]

#scores = ["precision", "recall_macro"]
scores = ["accuracy"]

print("# tuning hypter parmeter for %s" % score)
print()
    
clf_svm = GridSearchCV(SVC(random_state = 2), parameters, scoring=scores, verbose = 3, n_jobs=6, cv=5)
clf_svm.fit(X_train, y_train)
    
print("best parameters st found on devleopment set:")
print()
print(clf_svm.best_params_)
print()
print("Grid scores on development set:")
means = clf_svm.cv_results_["mean_test_score"]
stds = clf_svm.cv_results_["std_test_score"]
    

for mean, std, params in tqdm(zip(means, stds, clf_svm.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=svm_tf_igm)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_svm.predict(X_test)
print(classification_report(y_true, y_pred), file=svm_tf_igm)

print()

svm_tf_igm.close()





###### end gridsearchCV for SVM classifer #######

In [ ]:
##### gridsearchCV for naive_bayes classifer #######

rb_igm_tf = open('tf_igm_rb.txt', 'w')


parameters = [{"alpha": [00.00001, 0.0001, 0.001, 0.01, 0.1, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 8, 9, 10, 20, 40, 60, 80, 100, 150, 200, 300, 400, 500], "fit_prior": ["True", "False"]}]
# scores = ["precision", "recall_macro"]
#scores = ["accuracy"]
scores = make_scorer(accuracy_score)

print("# tuning hypter parmeter for %s" % scores)
print()

clf_rb = GridSearchCV(rb, parameters, scoring=scores, verbose=3, n_jobs=6, cv=5)
clf_rb.fit(X_train, y_train)

print("best parameters st found on devleopment set:")
print()
print(clf_rb.best_params_)
print()
print("Grid scores on development set:")
means = clf_rb.cv_results_["mean_test_score"]
stds = clf_rb.cv_results_["std_test_score"]

for mean, std, params in tqdm(zip(means, stds, clf_rb.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=rb_tf_igm)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_rb.predict(X_test)
print(classification_report(y_true, y_pred), file=rb_tf_igm)

print()

svm_tf_igm.close()


###### end gridsearchCV for naive_bayes classifer #######

In [ ]:

##### gridsearchCV for ramdon forest classifer #######

rf_igm_tf = open('tf_igm_rf.txt', 'w')

rf = RandomForestClassifier(random_state=2017)
parameters = [{"n_estimators": [4, 6, 8, 10, 15, 20, 40, 60, 80, 100, 150, 200, 300, 400, 500, 600, 800, 1000], "max_features": ['sqrt', 'log2'], "max_depth": [4, 6, 8, 10, 12, 16, 18, 20, 22, 24, 26, 28, 30], 'criterion': ['gini', 'entropy', 'log_loss']}]
scores = make_scorer(accuracy_score)

print("# tuning hypter parmeter for %s" % scores)
print()

clf_rf = GridSearchCV(rf, parameters, scoring=scores, verbose=3, n_jobs=6, cv=5)
clf_rf.fit(X_train, y_train)

print("best parameters st found on devleopment set:")
print()
print(clf_rf.best_params_)
print()
print("Grid scores on development set:")
means = clf_rf.cv_results_["mean_test_score"]
stds = clf_rf.cv_results_["std_test_score"]

for mean, std, params in tqdm(zip(means, stds, clf_rf.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=rf_tf_igm)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_rb.predict(X_test)
print(classification_report(y_true, y_pred), file=rf_tf_igm)

print()

rf_tf_igm.close()



##### end gridsearchCV for ramdon forest classifer #######


In [ ]:

##### gridsearchCV for Logistic regression classifer #######


lr_tf_igm = open('tf_igm_lr.txt', 'w')

lr = LogisticRegression(multi_class='multinomial', class_weight='balanced', random_state=2017)

c_range = [0.001, 0.01, 0.1, 1, 10, 100, 200]
max_i = [20, 50, 100, 200, 500, 1000]
parameters = [{"penalty": ["l1"], "C": c_range, "solver": ["saga"], 'max_iter': max_i},
              {"penalty": ["l2"], "C": c_range, "solver": ["saga"], 'max_iter': max_i},
              {"penalty": ["elasticnet"], "C": c_range, "solver": ["saga"], 'max_iter': max_i, 'l1_ratio': [0.5]}]

scores = make_scorer(accuracy_score)

print("# tuning hypter parmeter for %s" % scores)
print()

clf_lr = GridSearchCV(lr, parameters, scoring=scores, verbose=3, n_jobs=6, cv=5)
clf_lr.fit(X_train, y_train)

print("best parameters st found on devleopment set:")
print()
print(clf_lr.best_params_)
print()
print("Grid scores on development set:")
means = clf_lr.cv_results_["mean_test_score"]
stds = clf_lr.cv_results_["std_test_score"]

for mean, std, params in tqdm(zip(means, stds, clf_lr.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=lr_tf_igm)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_lr.predict(X_test)
print(classification_report(y_true, y_pred), file=lr_tf_igm)

print()

lr_tf_igm.close()




##### end gridsearchCV for Logistic regression classifer #######



In [ ]:


##### gridsearchCV for XGBoost classifer #######


xgb_tf_igm = open('tf_igm_xgb.txt', 'w')


xgb = XGBClassifier(random_state=2017)
parameters = [{"max_depth":[2, 4, 8, 12, 16, 18, 22, 26], "gamma":[0.1, 1, 2, 4, 8, 12, 16, 20, 24], "colsample_bytree": [0.1, 0.2, 0.4, 0.6],
               "min_child_weight":[1, 2, 4, 8, 10], "subsample":[0.1, 0.5, 0.7, 1]}]
# scores = ["precision", "recall_macro"]
#scores = ["accuracy"]
scores = make_scorer(accuracy_score)

print("# tuning hypter parmeter for %s" % scores)
print()

clf_xgb = GridSearchCV(xgb, parameters, scoring=scores, verbose=3, n_jobs=6, cv=3)
clf_xgb.fit(X_train, y_train)

print("best parameters st found on devleopment set:")
print()
print(clf_xgb.best_params_)
print()
print("Grid scores on development set:")
means = clf_xgb.cv_results_["mean_test_score"]
stds = clf_xgb.cv_results_["std_test_score"]

for mean, std, params in tqdm(zip(means, stds, clf_xgb.cv_results_["params"])):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params), file=xgb_tf_igm)

print()

print("Detailed classifciaiton reprot:")
print()
print("the model is trained on the full development set.")
print("the scores are computed on the full developement set")
print()
y_true, y_pred = y_test, clf_xgb.predict(X_test)
print(classification_report(y_true, y_pred), file=xgb_tf_igm)

print()

xgb_tf_igm.close()


##### end gridsearchCV for XGBoost classifer #######


